In [156]:
import pandas as pd
import ta
import matplotlib.pyplot as plt
import numpy as np

In [157]:
gaz = pd.read_csv('/Users/b23/Documents/GitHub/Reinforcement-Learning-Project/Dutch TTF Natural Gas Futures - Données Historiques (1).csv')
gaz['Date']=pd.to_datetime(gaz['Date'])
lists = ['Dernier', 'Ouv.', ' Plus Haut', 'Plus Bas', 'Vol.','Variation %']
for i in lists: 
    gaz[i] = gaz[i].str.replace(',', '.')
gaz.columns

/var/folders/lz/qtyfs34j79n27bt7y8vw2n140000gn/T/ipykernel_979/2509170913.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  gaz['Date']=pd.to_datetime(gaz['Date'])


Index(['Date', 'Dernier', 'Ouv.', ' Plus Haut', 'Plus Bas', 'Vol.',
       'Variation %'],
      dtype='object')

## Feature Engineering 

Technical Indicators

In [158]:
gaz['Dernier'] = pd.to_numeric(gaz['Dernier'], errors='coerce')

gaz['rsi'] = ta.momentum.rsi(gaz['Dernier'], window=14)
gaz['macd'] = ta.trend.macd_diff(gaz['Dernier'])
gaz['ema20'] = ta.trend.ema_indicator(gaz['Dernier'], window=20)
gaz['ema50'] = ta.trend.ema_indicator(gaz['Dernier'], window=50)

gaz

,Date,Dernier,Ouv.,Plus Haut,Plus Bas,Vol.,Variation %,rsi,macd,ema20,ema50
0,2024-03-15,27.050,26.255,27.085,26.000,0.08K,3.84%,NaN,NaN,NaN,NaN
1,2024-03-14,26.050,24.675,26.600,24.675,0.51K,4.60%,NaN,NaN,NaN,NaN
2,2024-03-13,24.905,25.100,25.425,24.555,0.24K,0.52%,NaN,NaN,NaN,NaN
3,2024-03-12,24.775,25.010,25.010,24.355,0.30K,-0.62%,NaN,NaN,NaN,NaN
4,2024-03-11,24.930,25.970,26.465,24.760,0.45K,-5.53%,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1612,2017-10-27,18.150,18.150,18.150,18.150,NaN,0.44%,34.299402,-0.128800,19.074238,19.420546
1613,2017-10-26,18.070,18.070,18.070,18.070,NaN,-0.22%,33.531057,-0.117551,18.978597,19.367584
1614,2017-10-25,18.110,18.110,18.110,18.110,NaN,0.84%,34.323259,-0.098296,18.895873,19.318267
1615,2017-10-24,17.960,17.960,17.960,17.960,NaN,-0.72%,32.747079,-0.086934,18.806742,19.265001


In [159]:
def convert_k_m_to_numeric(value):
    """
    Convert values with 'K' or 'M' suffix to float numbers.
    Args:
    - value: The string or numeric value to convert.
    
    Returns:
    - The converted value as float if 'K' or 'M' was found; otherwise, the original value.
    """
    if isinstance(value, str):  # Only process strings
        if value.endswith('K'):
            return float(value[:-1]) * 1e3
        elif value.endswith('M'):
            return float(value[:-1]) * 1e6
    return value

In [160]:
gaz['Vol.'] = gaz['Vol.'].apply(convert_k_m_to_numeric)

In [161]:
for column in gaz.columns:
    gaz[column] = pd.to_numeric(gaz[column], errors='coerce')

In [162]:
assert gaz.applymap(np.isreal).all().all(), "Non-numeric data found in the dataset."


/var/folders/lz/qtyfs34j79n27bt7y8vw2n140000gn/T/ipykernel_979/2367427009.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  assert gaz.applymap(np.isreal).all().all(), "Non-numeric data found in the dataset."


In [163]:
class FeatureEngineer:
    def __init__(self, data):
        self.data = data

    def calculate_technical_indicators(self):
        """
        Calculate specified technical indicators for the dataset.
        """
        self.data['rsi'] = ta.momentum.rsi(self.data['Dernier'], window=14)
        self.data['macd'] = ta.trend.macd_diff(self.data['Dernier'])
        self.data['ema20'] = ta.trend.ema_indicator(self.data['Dernier'], window=20)
        self.data['ema50'] = ta.trend.ema_indicator(self.data['Dernier'], window=50)

    def normalize_features(self):
        """
        Normalize features to have a similar scale.
        """
        from sklearn.preprocessing import MinMaxScaler
        scaler = MinMaxScaler()
        features = ['rsi', 'macd', 'ema20', 'ema50']  # Specify the features to normalize
        self.data[features] = scaler.fit_transform(self.data[features])

    def integrate_economic_indicators(self, economic_data):
        """
        Integrate economic indicators with market data.
        This function assumes economic_data is a DataFrame where columns are indicators and rows align with self.data's timeline.
        """
        self.data = pd.concat([self.data, economic_data], axis=1)

    def construct_feature_vector(self):
        """
        Combine all features into a single vector for each timestep.
        Assuming all necessary features are already columns in self.data,
        this function will return a numpy array representation of the DataFrame.
        """
        return self.data.values

# Example usage:
# Assuming 'gaz' is your DataFrame loaded with 'Dernier' prices and indexed by datetime
feature_engineer = FeatureEngineer(gaz)

feature_engineer.calculate_technical_indicators()  # Calculate RSI, MACD, EMA20, EMA50
# If you have economic data to integrate, load it and pass it to the integrate_economic_indicators method
# economic_data = pd.read_csv('path_to_your_economic_data.csv', index_col='date', parse_dates=True)
# feature_engineer.integrate_economic_indicators(economic_data)

feature_engineer.normalize_features()  # Normalize the features
features_vector = feature_engineer.construct_feature_vector()  # Get the feature vector

In [164]:
gaz

,Date,Dernier,Ouv.,Plus Haut,Plus Bas,Vol.,Variation %,rsi,macd,ema20,ema50
0,1710460800000000000,27.050,26.255,27.085,26.000,80.0,NaN,NaN,NaN,NaN,NaN
1,1710374400000000000,26.050,24.675,26.600,24.675,510.0,NaN,NaN,NaN,NaN,NaN
2,1710288000000000000,24.905,25.100,25.425,24.555,240.0,NaN,NaN,NaN,NaN,NaN
3,1710201600000000000,24.775,25.010,25.010,24.355,300.0,NaN,NaN,NaN,NaN,NaN
4,1710115200000000000,24.930,25.970,26.465,24.760,450.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1612,1509062400000000000,18.150,18.150,18.150,18.150,NaN,NaN,0.224810,0.475943,0.059504,0.069041
1613,1508976000000000000,18.070,18.070,18.070,18.070,NaN,NaN,0.213397,0.476425,0.059111,0.068779
1614,1508889600000000000,18.110,18.110,18.110,18.110,NaN,NaN,0.225165,0.477251,0.058771,0.068534
1615,1508803200000000000,17.960,17.960,17.960,17.960,NaN,NaN,0.201751,0.477738,0.058404,0.068270


## GYM Trading Environment

In [165]:
import numpy as np
import pandas as pd

class TradingEnvironment:
    def __init__(self, data, initial_balance=10000, transaction_cost=0.001):
        self.data = data
        self.state_space = data.shape[1]  # This should match the number of features used to represent a state
        self.action_space = 3  # For example: buy, sell,
        self.initial_balance = initial_balance
        self.transaction_cost = transaction_cost
        self.reset()

    def reset(self):
        self.current_step = 0
        self.balance = self.initial_balance
        self.portfolio_value = self.initial_balance
        self.done = False
        self.position = 0
        self.history = []  # To store trade history
        return self._next_observation()

    def _next_observation(self):
        return self.data.iloc[self.current_step]

    def step(self, action):
        # Simulate taking an action and returning the next state, reward, and done status
        
        # Ensure action is within a valid range
        action = np.clip(action, -1, 1)

        # Calculate the number of shares bought/sold based on the action
        delta_position = action * self.balance  # This assumes all-in on each action

        # Get the current price from the dataset to calculate changes in portfolio value
        current_price = self.data.iloc[self.current_step]['Dernier']  # Assuming 'close' is a column in your dataset
        next_step = min(self.current_step + 1, len(self.data) - 1)  # Ensure we don't go past the end of the dataset
        next_price = self.data.iloc[next_step]['Dernier']

        # Update position and balance
        change_in_value = delta_position * (next_price - current_price) / current_price
        self.balance += change_in_value - (abs(delta_position) * self.transaction_cost)
        self.portfolio_value = self.balance  # This could be more complex if managing multiple positions
        self.position += delta_position

        self.current_step = next_step

        # Check if we're at the end
        if self.current_step >= len(self.data) - 1:
            self.done = True

        # Here, the reward could be the change in portfolio value, or some other metric
        reward = change_in_value - (abs(delta_position) * self.transaction_cost)
        
        # Record this step
        self.history.append((self.current_step, self.position, self.portfolio_value, reward))

        return self._next_observation(), reward, self.done, {}

    def render(self):
        print("Step:", self.current_step)
        print("Balance:", self.balance)
        print("Position:", self.position)
        print("Portfolio Value:", self.portfolio_value)

    def run_backtest(self, policy):
        """
        Run a full backtest of the trading environment using the provided policy.
        The policy function should take a state as input and return an action.
        """
        self.reset()
        while not self.done:
            current_state = self._next_observation()
            action = policy(current_state)
            self.step(action)
        return self.history

# Example usage:

# Define a simple policy function (replace this with your PPO agent's policy for real testing)
def sample_policy(state):
    # This is a dummy policy that randomly decides to buy, hold, or sell
    return np.random.uniform(-1, 1)

# Assuming 'gaz_data' is your preprocessed DataFrame including 'Dernier' prices and any features
env = TradingEnvironment(gaz)
backtest_history = env.run_backtest(sample_policy)


## PPO algorithm

In [166]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch.nn.functional as F
from itertools import chain
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate
import numpy as np


In [181]:
def compute_gae(next_value, rewards, masks, values, gamma=0.99, tau=0.95):
    gae = 0
    returns = []
    for step in reversed(range(len(rewards))):
        delta = rewards[step] + gamma * next_value * masks[step] - values[step]
        gae = delta + gamma * tau * masks[step] * gae
        next_value = values[step]
        returns.insert(0, gae + values[step])
    return torch.tensor(returns)

class PolicyNetwork(nn.Module):
    def __init__(self, input_dims, n_actions):
        super(PolicyNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dims, 64)
        self.fc2 = nn.Linear(64, 64)
        self.mean_layer = nn.Linear(64, n_actions)
        self.variance_layer = nn.Linear(64, n_actions)
        self.n_actions = n_actions

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        mean = self.mean_layer(x)
        variance = torch.exp(self.variance_layer(x))  # Ensure variance is positive
        return mean, variance  

def update_policy(self, states, actions, rewards, next_states, dones):
    # Calculate advantages and discounted rewards
    # Note: This will require implementing or calling additional methods to calculate these values
    advantages, discounted_rewards = self.calculate_advantages(rewards, states, next_states, dones)
    
    # Convert lists to numpy arrays for processing
    states = np.array(states)
    actions = np.array(actions)
    advantages = np.array(advantages)
    discounted_rewards = np.array(discounted_rewards)

    # Update actor
    with tf.GradientTape() as tape:
        # Calculate loss for the actor
        # Note: You'll need to implement the logic to calculate the actor's loss based on the current policy and advantages
        actor_loss = self.calculate_actor_loss(states, actions, advantages)
    actor_grads = tape.gradient(actor_loss, self.actor.trainable_variables)
    self.actor.optimizer.apply_gradients(zip(actor_grads, self.actor.trainable_variables))

    # Update critic
    with tf.GradientTape() as tape:
        # Calculate loss for the critic
        # Note: Implement logic to calculate the critic's loss based on the difference between discounted rewards and the critic's value predictions
        critic_loss = self.calculate_critic_loss(states, discounted_rewards)
    critic_grads = tape.gradient(critic_loss, self.critic.trainable_variables)
    self.critic.optimizer.apply_gradients(zip(critic_grads, self.critic.trainable_variables))


In [186]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.optimizers import legacy as optimizers_legacy

class PPOAgent:
    def __init__(self, state_size, action_size, action_bound):
        self.state_size = state_size
        self.action_size = action_size
        self.action_bound = action_bound
        self.gamma = 0.99
        self.lambda_ = 0.95
        self.epsilon = 0.2
        self.lr_actor = 1e-4
        self.lr_critic = 2e-4
        
        # Build actor and critic models within the constructor
        self.actor = self.build_actor()
        self.critic = self.build_critic()

    def build_actor(self):
        """Builds the actor model."""
        state_input = Input(shape=(self.state_size,))
        dense1 = Dense(64, activation='relu')(state_input)
        dense2 = Dense(64, activation='relu')(dense1)
        action_output = Dense(self.action_size, activation='tanh')(dense2)
        action_output = self.action_bound * action_output
        
        model = Model(inputs=state_input, outputs=action_output)
        model.compile(optimizer=optimizers_legacy.Adam(learning_rate=self.lr_actor))
        return model



    def build_critic(self):
        """Builds the critic model."""
        state_input = Input(shape=(self.state_size,))
        action_input = Input(shape=(self.action_size,))
        concat = Concatenate()([state_input, action_input])
        
        dense1 = Dense(64, activation='relu')(concat)
        dense2 = Dense(64, activation='relu')(dense1)
        q_value_output = Dense(1, activation='linear')(dense2)
        
        model = Model(inputs=[state_input, action_input], outputs=q_value_output)
        model.compile(optimizer=optimizers_legacy.Adam(learning_rate=self.lr_critic))
        return model
    

    def preprocess_state(self, state):
        # Convert Timestamps to float (Unix time) if present
        if isinstance(state, pd.DataFrame) or isinstance(state, pd.Series):
            state = state.apply(lambda x: x.timestamp() if isinstance(x, pd.Timestamp) else x)
        elif isinstance(state, np.ndarray) and state.dtype == 'object':
            state = np.array([x.timestamp() if isinstance(x, pd.Timestamp) else x for x in state], dtype=np.float32)
        return state

    def predict_action(self, state):
        # Preprocess the state to ensure numeric representation
        state = self.preprocess_state(state)
        
        # Reshape the state to match the expected input dimensions of the model
        state = np.array(state, dtype=np.float32)
        state = np.expand_dims(state, axis=0)
                
        # Predict action probabilities and choose the action with the highest probability
        action_probabilities = self.actor.predict(state)
        return np.argmax(action_probabilities)
    
    

# Define the environment and PPO agent parameters
state_size = 100  # Assume 100 features for the state
action_size = 1  # Buy/sell quantity as a single action
action_bound = 1  # Action limit (e.g., normalized quantity between -1 and 1)

# Instantiate the PPOAgent
agent = PPOAgent(state_size=env.state_space, action_size=env.action_space, action_bound=1)

# Now you can access actor and critic through the agent object
actor_model = agent.actor
critic_model = agent.critic

# Optionally, print summaries to verify the models
actor_model.summary()
critic_model.summary()


Model: "model_99"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_149 (InputLayer)      [(None, 11)]              0         
                                                                 
 dense_297 (Dense)           (None, 64)                768       
                                                                 
 dense_298 (Dense)           (None, 64)                4160      
                                                                 
 dense_299 (Dense)           (None, 3)                 195       
                                                                 
 tf.math.multiply_50 (TFOpL  (None, 3)                 0         
 ambda)                                                          
                                                                 
Total params: 5123 (20.01 KB)
Trainable params: 5123 (20.01 KB)
Non-trainable params: 0 (0.00 Byte)
________________________

## Training

In [183]:
def train_agent(env, agent, episodes=1000):
    for episode in range(episodes):
        state = env.reset()
        done = False
        while not done:
            action = agent.predict_action(state)  # Predict the action for the current state
            next_state, reward, done, _ = env.step(action)  # Take the action in the environment
            agent.update_policy(state, action, reward, next_state, done)  # Update the policy
            state = next_state
        print(f"Episode {episode + 1}: Complete")
    print("Training complete")


In [184]:
def run_simulation(env, agent):
    state = env.reset()
    done = False
    total_pnl = 0
    print("Date\t\tAction\tReward\tPortfolio Value")

    while not done:
        action = agent.predict_action(state)
        next_state, reward, done, _ = env.step(action)

        # Action interpretation for logging: -1 (Sell), 1 (Buy), 0 (Hold)
        action_str = "Buy" if action > 0 else "Sell" if action < 0 else "Hold"

        # Assuming 'date' is part of the environment's state
        date = state.index[env.current_step].strftime('%Y-%m-%d')
        print(f"{date}\t{action_str}\t{reward:.2f}\t{env.portfolio_value:.2f}")

        total_pnl += reward
        state = next_state

    print(f"Final PnL: {total_pnl:.2f}")


In [185]:
env = TradingEnvironment(gaz)  # Ensure 'gaz' is your DataFrame properly prepared
agent = PPOAgent(state_size=env.state_space, action_size=env.action_space, action_bound=1)  # Adjust parameters as needed

# Proceed with training and running the simulation
train_agent(env, agent)
run_simulation(env, agent)

1/1 [==============================] - 0s 26ms/step


AttributeError: 'PPOAgent' object has no attribute 'update_policy'